In [31]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import acquire as a
import prepare as p

In [24]:
df = a.new_titanic_data()
df

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,None,Southampton,1
887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.4500,S,Third,None,Southampton,0
889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


In [25]:
train, validate, test = p.split_data(df)
train.shape, validate.shape, test.shape

((498, 13), (214, 13), (179, 13))

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 583 to 744
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  498 non-null    int64  
 1   survived      498 non-null    int64  
 2   pclass        498 non-null    int64  
 3   sex           498 non-null    object 
 4   age           401 non-null    float64
 5   sibsp         498 non-null    int64  
 6   parch         498 non-null    int64  
 7   fare          498 non-null    float64
 8   embarked      497 non-null    object 
 9   class         498 non-null    object 
 10  deck          106 non-null    object 
 11  embark_town   497 non-null    object 
 12  alone         498 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 54.5+ KB


In [36]:
train = train.drop(columns=['deck','embarked','class'])

KeyError: "['deck' 'embarked' 'class'] not found in axis"

In [37]:
train = pd.get_dummies(train, drop_first = True)

In [40]:
imp = SimpleImputer(strategy="mean")
train = imp.fit_transform(train)
type(train)

numpy.ndarray

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [ ]:
X_train, y_train = train.drop(columns='survived'), train.survived
X_validate, y_validate = validate.drop(columns='survived'), validate.survived
X_test, y_test = test.drop(columns='survived'), test.survived

In [ ]:
y_train.value_counts()

Since most passengers did not survive ('0'), our baseline prediction will be 'survive == 0'

In [ ]:
# 1. Create the object (model)
baseline = DummyClassifier(strategy='constant', constant=0) # baseline_prediction, we're predicting no one survives
# 2. Fit the object
baseline.fit(X_train, y_train)

In [ ]:
accuracy = round(baseline.score(X_train, y_train), 2)
print(f'Baseline Training accuracy: {100*accuracy}%')

In [ ]:
X_train = pd.get_dummies(X_train, drop_first = True)
X_train

### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [ ]:
clf = DecisionTreeClassifier(max_depth=5, random_state=123)
clf = clf.fit(X_train, y_train)

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

### 5. Run through steps 2-4 using a different max_depth value.

### 6. Which model performs better on your in-sample data?

### 7. Which model performs best on your out-of-sample data, the validate set?